In [1]:
!pip install implicit --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 54.2 MB/s eta 0:00:00


In [2]:
from datetime import timedelta
import polars as pl
import implicit

In [3]:
# train_data = pl.read_parquet(f'/kaggle/input/shoping-data/train_data.pq')
# train_data

In [4]:
import pandas as pd

samples = pd.read_csv("/kaggle/input/shoping-data/sample_submission (7).csv")

samples

,user_id,item_id
0,247446,140716
1,247446,140716
2,247446,140716
3,247446,140716
4,247446,140716
...,...,...
5864595,244238,140716
5864596,244238,140716
5864597,244238,140716
5864598,244238,140716


In [5]:
samples.shape, samples['user_id'].nunique()

((5864600, 2), 293230)

In [6]:
samples['user_id'].value_counts()

user_id
247446     20
1402486    20
2627563    20
918952     20
78983      20
           ..
742443     20
349469     20
260502     20
653912     20
244238     20
Name: count, Length: 293230, dtype: int64

In [7]:
import pandas as pd
from scipy.sparse import coo_matrix
import numpy as np

In [8]:
# # Уникальные пользователи и товары
# user_ids = train["user_id"].unique()
# item_ids = train["item_id"].unique()

# # Словари индексов
# user2idx = {u: i for i, u in enumerate(user_ids)}
# item2idx = {i: j for j, i in enumerate(item_ids)}

# idx2user = {i: u for u, i in user2idx.items()}
# idx2item = {j: i for i, j in item2idx.items()}

# # Применяем к train
# train["user_idx"] = train["user_id"].map(user2idx)
# train["item_idx"] = train["item_id"].map(item2idx)

In [9]:
# print("Users:", len(user2idx), " Items:", len(item2idx))
# print(train[["user_id", "user_idx", "item_id", "item_idx"]].head())

In [10]:
# rows = train["user_idx"].values
# cols = train["item_idx"].values
# data = np.ones(len(train), dtype=np.float32)

# user_item_matrix = coo_matrix(
#     (data, (rows, cols)),
#     shape=(len(user2idx), len(item2idx))
# )

In [11]:
# model = implicit.als.AlternatingLeastSquares(
#     factors=50,
#     regularization=0.01,
#     iterations=20,
#     random_state=42
# )

# # implicit ожидает item-user матрицу
# user_item_matrix_csr = user_item_matrix.tocsr()
# item_user_matrix = user_item_matrix_csr.T

# print("Обучаем ALS...")
# model.fit(item_user_matrix)
# print("✅ Модель обучена!")

In [12]:
# preds = []

# popular_items = train["item_id"].value_counts().index[:20].tolist()

# for user_id in tqdm(sample["user_id"], desc="Predicting"):
#     if user_id not in user2idx:
#         print(f'user_id not in user2idx: {user_id}')
#         rec_items = popular_items
#     else:
#         try:
#             user_idx = user2idx[user_id]
#         except:
#             print(user_id)
#             break
#         user_row = user_item_matrix_csr[user_idx]

#         rec_indices, _ = model.recommend(
#             userid=0,
#             user_items=user_row,
#             N=20,
#             filter_already_liked_items=True
#         )
#         rec_items = [idx2item[i] for i in rec_indices]
    
#     preds.append({
#         "user_id": user_id,
#         "item_ids": " ".join(map(str, rec_items))
#     })

In [13]:
# print(rec_indices)
# print(len(idx2item))
# print(max(rec_indices))

In [14]:
# import pandas as pd

# # Загружаем данные через Pandas
# train = pd.read_parquet('/kaggle/input/shoping-data/train_data.pq')

In [15]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

# Загружаем данные
train = pd.read_parquet("/kaggle/input/shoping-data/train_data.pq")

# Перенумерация пользователей и товаров
user_ids = train['user_id'].unique()
item_ids = train['item_id'].unique()

user2idx = {u: i for i, u in enumerate(user_ids)}
item2idx = {i: j for j, i in enumerate(item_ids)}

idx2user = {i: u for u, i in user2idx.items()}
idx2item = {j: i for i, j in item2idx.items()}

# Создаём матрицу напрямую, без новых столбцов
rows = train['user_id'].map(user2idx)
cols = train['item_id'].map(item2idx)
data = np.ones(len(train), dtype=np.float32)

user_item_matrix = coo_matrix(
    (data, (rows, cols)),
    shape=(len(user2idx), len(item2idx))
)

user_item_matrix_csr = user_item_matrix.tocsr()
item_user_matrix = user_item_matrix_csr.T

In [16]:
user_item_matrix_csr
# <Compressed Sparse Row sparse matrix of dtype 'float32'
# 	with 8413765 stored elements and shape (2682603, 740651)>

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 8413765 stored elements and shape (2682603, 740651)>

In [17]:
print(train.shape)
print(train['user_id'].nunique())
# (8777975, 5)
# 2682603

(8777975, 3)
2682603


In [18]:
import implicit

# ALS для implicit feedback
model = implicit.als.AlternatingLeastSquares(
    factors=50,           # размер латентного пространства
    regularization=0.01,  # регуляризация
    iterations=20,
    random_state=42
)

print("Обучаем ALS...")
model.fit(item_user_matrix)  # матрица item x user
print("Модель обучена!")

Обучаем ALS...


/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()
/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
/usr/local/lib/python3.11/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.27223896980285645 seconds
  warnings.warn(

  0%|          | 0/20 [00:00<?, ?it/s]

Модель обучена!


In [19]:
preds = []

# Топ-20 популярных товаров на случай новых пользователей
popular_items = train["item_id"].value_counts().index[:20].tolist()

from tqdm import tqdm

for user_id in tqdm(samples["user_id"].unique(), desc="Predicting"):
    rec_items = []
    if user_id not in user2idx:
        # Для редких пользователей — просто топ популярных
        rec_items = popular_items
    else:
        user_idx = user2idx[user_id]
        user_row = user_item_matrix_csr[user_idx]

        # Получаем топ-20 рекомендаций
        rec_indices, _ = model.recommend(
            userid=0,  # индекс строки user_row
            user_items=user_row,
            N=20,
            filter_already_liked_items=True
        )
        
        for i in rec_indices:
            if i in idx2item:
                rec_items.append(idx2item[i])
            else:
                # Если индекс неизвестен, используем топ популярных
                rec_items.append(popular_items[0])
        for item_id in rec_items:
            preds.append((user_id, item_id))

Predicting: 100%|██████████| 293230/293230 [2:33:59<00:00, 31.74it/s]


In [20]:
my_submition = pd.DataFrame(preds, columns=['user_id', 'item_id'])

In [21]:
my_submition['user_id'].nunique()

293230

In [22]:
my_submition['user_id'].value_counts()

user_id
247446     20
1402486    20
2627563    20
918952     20
78983      20
           ..
742443     20
349469     20
260502     20
653912     20
244238     20
Name: count, Length: 293230, dtype: int64

In [23]:
 # Сохраняем
my_submition.to_csv("submission.csv", index=False)